In [1]:
from langchain.vectorstores import Chroma

from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.tools import Tool

from langchain.document_loaders import DirectoryLoader, TextLoader

from langchain.memory import ConversationBufferMemory

from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder

from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor

from langchain.chains import RetrievalQA

#import env
import os
import sys

In [2]:
OPENAI_API_KEY = ""

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
loader=TextLoader('analects_raw.txt',encoding='utf-8')
data=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=10)
splits=text_splitter.split_documents(data)
embedding=OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])
vectordb=Chroma.from_documents(
    documents=splits,embedding=embedding,collection_name='personal-data'
)
llm=ChatOpenAI(temperature=0)
personal_data=RetrievalQA.from_chain_type(
    llm=llm, chain_type='stuff',retriever=vectordb.as_retriever()
)
personal_data_tool=Tool(
    name='PersonalData',
    func=personal_data.run,
    description='이건 논어야'
)
memory_key='history'
memory=ConversationBufferMemory(memory_key=memory_key,return_messages=True)
system_message=SystemMessage(
    content=(
        '너가 대답할 수 있는 최고의 대답을 해줘'
        '논어를 적극 활용해서 논어랑 가장 관련성 높은 대답을 해줘'
        '논어에 써있는 사례도 같이 말해줘'
        '논어의 말투처럼 말해줘'
    )
)

In [ ]:
prompt=OpenAIFunctionsAgent.create_prompt(
    system_message=system_message,
    extra_prompt_messages=[MessagesPlaceholder(variable_name=memory_key)]
)
tools=[personal_data_tool]
agent=OpenAIFunctionsAgent(llm=llm,tools=tools,prompt=prompt)
agent_executor=AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)

In [ ]:
result=agent_executor({'input': '공자님, 삶이 힘들고 고달픕니다. 공부도 하기싫습니다. 어떻게 해야 좋을까요?'})
prompt=None